# Capstone 2 = Data Wrangling
**Project:** Modeling USD/JPY movements with macroeconomic drivers 

**Author:** Matt Snyder

The goal of this notebook is to collect, organize, and clean all macroeconomic time-series needed to model USD/JPY at a monthly frequency. Aside from GDP, the datasets are already organized by the 1st of the month and GDP will need to be forward filled and set at the beginning of the month by converting to M and to datetime to timestamp.

In [1]:
import pandas as pd
import numpy as np
import os

# 1. Data Collection

Load all raw datasets from `data_raw\` folder.

In [2]:
usd_jpy = pd.read_csv("data_raw/usd_jpy.csv")
us_fed_funds = pd.read_csv("data_raw/us_fed_funds_rate.csv")
jp_call_rate = pd.read_csv("data_raw/jp_st_int_rate.csv")
us_cpi = pd.read_csv("data_raw/us_cpi_index.csv")
jp_cpi = pd.read_csv("data_raw/jp_cpi_index.csv")
brent_oil = pd.read_csv("data_raw/brent_oil_prices.csv")
us_gdp = pd.read_csv("data_raw/us_gdp.csv")
boj_cpi = pd.read_excel("data_raw/boj_cpi_breakdown.xlsx")
dollar_indx = pd.read_csv("data_raw/us_dollar_index.csv")

In [3]:
os.getcwd()

'C:\\Users\\msnyd\\code\\springBoard\\USD_JPY_Capstone\\Data_Wrangling'

# 2. Data Label/Definition Checks

Check:
-Col Names
-Dtypes
-Time coverage checks

In [4]:
for name, df in {
    "USD JPY": usd_jpy,
    "US Fed Funds": us_fed_funds,
    "JP Call Rate": jp_call_rate,
    "US CPI": us_cpi,
    "JP CPI": jp_cpi,
    "Brent Oil": brent_oil,
    "US GDP": us_gdp,
    "USD Index": dollar_indx
}.items():
    print("\n---", name, "---")
    print(df.info())
    print(df.head())


--- USD JPY ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  659 non-null    object 
 1   DEXJPUS           658 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.4+ KB
None
  observation_date  DEXJPUS
0       1971-01-01   358.02
1       1971-02-01   357.55
2       1971-03-01   357.52
3       1971-04-01   357.50
4       1971-05-01   357.41

--- US Fed Funds ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856 entries, 0 to 855
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  856 non-null    object 
 1   FEDFUNDS          856 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.5+ KB
None
  observation_date  FEDFUNDS
0       1954-07-01      0.80
1       1954-08-01      1.22
2     

In [5]:
usd_jpy.head()

,observation_date,DEXJPUS
0,1971-01-01,358.02
1,1971-02-01,357.55
2,1971-03-01,357.52
3,1971-04-01,357.50
4,1971-05-01,357.41


*sanityCheck*

In [6]:
usd_jpy['observation_date'].dtype

dtype('O')

In [7]:
usd_jpy.rename(columns={'observation_date': 'DATE'}, inplace=True)
dollar_indx.rename(columns={'observation_date': 'DATE'}, inplace=True)
us_fed_funds.rename(columns={'observation_date': 'DATE'}, inplace=True)
jp_call_rate.rename(columns={'observation_date': 'DATE'}, inplace=True)
us_cpi.rename(columns={'observation_date': 'DATE'}, inplace=True)
jp_cpi.rename(columns={'observation_date': 'DATE'}, inplace=True)
brent_oil.rename(columns={'observation_date': 'DATE'}, inplace=True)
us_gdp.rename(columns={'observation_date': 'DATE'}, inplace=True)

In [8]:
boj_cpi.columns

Index([ 'Unnamed: 0',  'Unnamed: 1',  'Unnamed: 2',  'Unnamed: 3',
        'Unnamed: 4',  'Unnamed: 5',  'Unnamed: 6',  'Unnamed: 7',
                   1,             2,             3,             4,
                   5,             6,             7,             8,
                   9,            10,            11,            12,
                  13,            14,            15,            16,
                  17,            18,            19,            20,
                  21,            22,            23,            24,
                  25,            26, 'Unnamed: 34', 'Unnamed: 35',
                  27,            28,            29,            30,
                  31,            32,            33,            34,
                  35,            36,            37,            38,
                  39,            40,            41,            42,
                  43,            44,            45,            46,
                  47,            48,            49,           

In [9]:
datasets = [usd_jpy, us_fed_funds, jp_call_rate, us_cpi, jp_cpi, brent_oil, us_gdp, dollar_indx]

for df in datasets:
    df['DATE'] = pd.to_datetime(df['DATE'])

**Confirm that `DATE` dtype has changed**

In [10]:
usd_jpy['DATE'].dtype

dtype('<M8[ns]')

In [11]:
brent_oil.head()

,DATE,DCOILBRENTEU
0,1987-06-01,18.86
1,1987-07-01,19.86
2,1987-08-01,18.98
3,1987-09-01,18.31
4,1987-10-01,18.76


In [12]:
for df, name in zip([usd_jpy, us_fed_funds, jp_call_rate, us_cpi, jp_cpi, brent_oil, dollar_indx],
                    ['USDJPY', 'US Fed Funds', 'JP Call Rate', 'US CPI', 'JP CPI', 'Brent Oil', 'USD Index']
                   ):
    print (name, df['DATE'].dt.day.unique()[:5])

USDJPY [1]
US Fed Funds [1]
JP Call Rate [1]
US CPI [1]
JP CPI [1]
Brent Oil [1]
USD Index [1]


In [13]:
(usd_jpy['DATE'].diff().unique())[:5]

<TimedeltaArray>
[NaT, '31 days', '28 days', '30 days', '29 days']
Length: 5, dtype: timedelta64[ns]

# 3. Standardize Data
&&

**Improve Columns Names**

In [14]:
usd_jpy.rename(columns={'DEXJPUS': 'usd_jpy'}, inplace=True)
us_fed_funds.rename(columns={'FEDFUNDS': 'us_fed_fund_rate'}, inplace=True)
jp_call_rate.rename(columns={'IRSTCI01JPM156N': 'jp_call_rate'}, inplace=True)
us_cpi.rename(columns={'CPIAUCSL': 'us_cpi_index'}, inplace=True)
jp_cpi.rename(columns={'JPNCPIALLMINMEI': 'jp_cpi_index'}, inplace=True)
brent_oil.rename(columns={'DCOILBRENTEU': 'brent_oil_price'}, inplace=True)
dollar_indx.rename(columns={'DTWEXBGS': 'us_dollar_index'}, inplace=True)
us_gdp.rename(columns={'GDP': 'us_gdp'}, inplace=True)

In [15]:
dfs = {
    "USD JPY": usd_jpy,
    "US Fed Funds": us_fed_funds,
    "JP Call Rate": jp_call_rate,
    "US CPI": us_cpi,
    "JP CPI": jp_cpi,
    "Brent Oil": brent_oil,
    "Dollar Index": dollar_indx,
    "US GDP": us_gdp
}

for name, df in dfs.items():
    print(name, df.columns)

USD JPY Index(['DATE', 'usd_jpy'], dtype='object')
US Fed Funds Index(['DATE', 'us_fed_fund_rate'], dtype='object')
JP Call Rate Index(['DATE', 'jp_call_rate'], dtype='object')
US CPI Index(['DATE', 'us_cpi_index'], dtype='object')
JP CPI Index(['DATE', 'jp_cpi_index'], dtype='object')
Brent Oil Index(['DATE', 'brent_oil_price'], dtype='object')
Dollar Index Index(['DATE', 'us_dollar_index'], dtype='object')
US GDP Index(['DATE', 'us_gdp'], dtype='object')


# 4. Align GDP dates with rest of dataset so all sets can be merged - 



**Need to forward fill missing GDP data, as it is run quarterly. Merge datasets on `DATE`**

In [16]:
us_gdp_m = (
    us_gdp
    .set_index('DATE')
    .resample('M')
    .ffill()
    .reset_index()
)

C:\Users\msnyd\AppData\Local\Temp\ipykernel_24352\856989388.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M')


In [17]:
us_gdp.head(), us_gdp_m.head()

(        DATE   us_gdp
 0 1947-01-01  243.164
 1 1947-04-01  245.968
 2 1947-07-01  249.585
 3 1947-10-01  259.745
 4 1948-01-01  265.742,
         DATE   us_gdp
 0 1947-01-31  243.164
 1 1947-02-28  243.164
 2 1947-03-31  243.164
 3 1947-04-30  245.968
 4 1947-05-31  245.968)

In [18]:
us_gdp_m['DATE'].dt.day.unique()

array([31, 28, 30, 29], dtype=int32)

**Currently GDP is dated at the end of the month, however it needs to be placed at the beginning to match the other datasets so they can be merged**

In [19]:
us_gdp_m['DATE'] = us_gdp_m['DATE'].dt.to_period('M').dt.to_timestamp()

In [20]:
us_gdp_m['DATE'].head()

0   1947-01-01
1   1947-02-01
2   1947-03-01
3   1947-04-01
4   1947-05-01
Name: DATE, dtype: datetime64[ns]

In [21]:
us_gdp_m['DATE'].dt.day.unique()

array([1], dtype=int32)

**Need to merge on DATE**

In [22]:
merged = usd_jpy.copy()
merged = merged.merge(us_fed_funds, on='DATE', how='left')
merged = merged.merge(jp_call_rate, on='DATE', how='left')
merged = merged.merge(us_cpi, on='DATE', how='left')
merged = merged.merge(jp_cpi, on='DATE', how='left')
merged = merged.merge(brent_oil, on='DATE', how='left')
merged = merged.merge(us_gdp_m, on='DATE', how='left')
merged = merged.merge(dollar_indx, on='DATE', how='left')

In [23]:
merged.head()

,DATE,usd_jpy,us_fed_fund_rate,jp_call_rate,us_cpi_index,jp_cpi_index,brent_oil_price,us_gdp,us_dollar_index
0,1971-01-01,358.02,4.14,NaN,39.9,32.78188,NaN,1135.156,NaN
1,1971-02-01,357.55,3.72,NaN,39.9,32.78188,NaN,1135.156,NaN
2,1971-03-01,357.52,3.71,NaN,40.0,32.88369,NaN,1135.156,NaN
3,1971-04-01,357.50,4.16,NaN,40.1,33.29091,NaN,1156.271,NaN
4,1971-05-01,357.41,4.63,NaN,40.3,33.39272,NaN,1156.271,NaN


In [24]:
merged.tail()

,DATE,usd_jpy,us_fed_fund_rate,jp_call_rate,us_cpi_index,jp_cpi_index,brent_oil_price,us_gdp,us_dollar_index
654,2025-07-01,147.20,4.33,0.478,322.132,NaN,71.04,NaN,120.5266
655,2025-08-01,147.48,4.33,0.477,323.364,NaN,67.87,NaN,120.9844
656,2025-09-01,147.86,4.22,0.477,324.368,NaN,67.99,NaN,120.4534
657,2025-10-01,151.35,4.09,NaN,NaN,NaN,64.54,NaN,121.1712
658,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
merged.shape

(659, 9)

**Ensure dataset is sorted by date**

## 5. Data Quality Checks

**Find missing values && conducting initial cleaning**

In [26]:
merged.isnull().sum()

DATE                  0
usd_jpy               1
us_fed_fund_rate      1
jp_call_rate        176
us_cpi_index          2
jp_cpi_index         53
brent_oil_price     198
us_gdp                7
us_dollar_index     421
dtype: int64

**We have several null values due to the datasets being recorded from different starting years. I will choose the starting year 2006, as that's when the dollar index started recording values and is most likely one of the bigger predictors or benchmarks that I can use to analyze JPY fluctuations.**

In [27]:
merged.duplicated(subset='DATE').sum()

np.int64(0)

**Check Data Range**

In [34]:
merged.sort_values('DATE', inplace = True)

In [28]:
merged['DATE'].min(), merged['DATE'].max()

(Timestamp('1971-01-01 00:00:00'), Timestamp('2025-11-01 00:00:00'))

In [29]:
merged_model = merged[merged['DATE'] >= '2006-01-01'].copy()

In [30]:
merged_model.head()

,DATE,usd_jpy,us_fed_fund_rate,jp_call_rate,us_cpi_index,jp_cpi_index,brent_oil_price,us_gdp,us_dollar_index
420,2006-01-01,115.48,4.29,0.001,199.3,96.92033,62.99,13599.160,100.0000
421,2006-02-01,117.86,4.49,0.001,199.4,96.61491,60.21,13599.160,100.2112
422,2006-03-01,117.28,4.59,0.002,199.7,96.81853,62.06,13599.160,100.4281
423,2006-04-01,117.07,4.79,0.002,200.7,97.02214,70.26,13753.424,99.7435
424,2006-05-01,111.73,4.94,0.020,201.3,97.32757,69.78,13753.424,97.5118


In [31]:
merged_model.isnull().sum()

DATE                 0
usd_jpy              1
us_fed_fund_rate     1
jp_call_rate         2
us_cpi_index         2
jp_cpi_index        53
brent_oil_price      1
us_gdp               7
us_dollar_index      1
dtype: int64

**Finding missing values and doing some initial cleaning. As seen below, DXY starts in 2006 and as it's the benchmark we are trying to analyze the JPY against, so I've decided to use 2006 as the starting year. As seen below, we still have missing values.**

**The first loop shows that there are no null values in the beginning (2006); however, the second loop locates where the missing values are. As shown below, our #1 bottleneck is JP_CPI (ending in 2021).**

**Will either end my dataset in 2021, or find Bank of Japan's (BOJ) official current data, or drop Japan CPI if justified (highly doubtful this will be dropped).**

In [32]:
for i in merged_model.columns:
    if i != 'DATE':
        first_valid = merged_model.loc[merged_model[i].notnull(), 'DATE'].min()
        print(i, first_valid)

usd_jpy 2006-01-01 00:00:00
us_fed_fund_rate 2006-01-01 00:00:00
jp_call_rate 2006-01-01 00:00:00
us_cpi_index 2006-01-01 00:00:00
jp_cpi_index 2006-01-01 00:00:00
brent_oil_price 2006-01-01 00:00:00
us_gdp 2006-01-01 00:00:00
us_dollar_index 2006-01-01 00:00:00


In [33]:
for i in merged_model.columns:
    if i != 'DATE':
        first_valid = merged_model.loc[merged_model[i].notnull(), 'DATE'].max()
        print(i, first_valid)

usd_jpy 2025-10-01 00:00:00
us_fed_fund_rate 2025-10-01 00:00:00
jp_call_rate 2025-09-01 00:00:00
us_cpi_index 2025-09-01 00:00:00
jp_cpi_index 2021-06-01 00:00:00
brent_oil_price 2025-10-01 00:00:00
us_gdp 2025-04-01 00:00:00
us_dollar_index 2025-10-01 00:00:00


**Save File**

In [35]:
os.makedirs('data_processed', exist_ok=True)
merged.to_csv('data_processed/macro_montly_merged.csv', index=False)